<a href="https://colab.research.google.com/github/Katonokatono/Suicide/blob/model/Transformers_Mildred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import numpy as np    
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
!pip install wandb
!wandb login 
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 180 kB 49.8 MB/s 
     |████████████████████████████████| 139 kB 48.3 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=6134c6db10396d266692c85d0734813739df7ebe8b49ea08d5237a74c93a2a4e
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=2b40a9a6acbbe6328a6fd74940a0f0d55df2d7d7747d7b8d8d0d0a611e41775d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wan

In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 101 kB/s 
     |████████████████████████████████| 6.8 MB 34.9 MB/s 
     |████████████████████████████████| 981 kB 35.6 MB/s 
     |████████████████████████████████| 263 kB 46.6 MB/s 
     |████████████████████████████████| 1.9 MB 39.2 MB/s 
     |████████████████████████████████| 1.2 MB 36.8 MB/s 
     |████████████████████████████████| 468 kB 42.1 MB/s 
     |████████████████████████████████| 895 kB 48.5 MB/s 
     |████████████████████████████████| 3.3 MB 45.0 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=848c5d61321f40d1063029d21cd1e13bef9046e754e30737752a97d715914a2d
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=1c0a9a346b7f9d6cb95c7be2eb86bdd9fae3d763377d5c265586b6bc932a99cd
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [5]:
#initialise wandb
%env WANDB_PROJECT='gpt3, entity='alpha-coders')

env: WANDB_PROJECT='gpt3, entity='alpha-coders')


In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [8]:
import ktrain
from ktrain import text
import pandas as pd

In [10]:
data = pd.read_csv('/content/balanced_data.csv')

In [11]:
data.head()

,Unnamed: 0,Unnamed: 0.1,friends_count,followers_count,statuses_count,created_at,retweet_count,label,town,clean_text
0,2864,133,424,55,412,9/13/2021 1:47,0,thoughts,nyeri,go torture deserve get bully since yr old body...
1,3350,624,3438,1562,9614,9/9/2021 4:07,6,thoughts,majengo,remember suicide thought thought act get contr...
2,2864,133,424,55,412,9/13/2021 1:47,0,thoughts,nyeri,go torture deserve get bully since yr old body...
3,3151,422,277,98,729,9/10/2021 8:13,0,thoughts,nairobi,suicide squad exist worldsuicidepreventionday ...
4,3259,533,5486,48711,295173,9/10/2021 4:09,0,thoughts,nairobi,people rarely commit suicide want die want sto...


In [12]:
# creating a dataframe with the tweet and target column
data_df= data[['clean_text','label']]
data_df.head()

,clean_text,label
0,go torture deserve get bully since yr old body...,thoughts
1,remember suicide thought thought act get contr...,thoughts
2,go torture deserve get bully since yr old body...,thoughts
3,suicide squad exist worldsuicidepreventionday ...,thoughts
4,people rarely commit suicide want die want sto...,thoughts


In [13]:
# Defining our X and Y varaibles
X = data_df['clean_text']
y= data_df['label']
print(X.head())
print('/n')
print(y.head())

0    go torture deserve get bully since yr old body...
1    remember suicide thought thought act get contr...
2    go torture deserve get bully since yr old body...
3    suicide squad exist worldsuicidepreventionday ...
4    people rarely commit suicide want die want sto...
Name: clean_text, dtype: object
/n
0    thoughts
1    thoughts
2    thoughts
3    thoughts
4    thoughts
Name: label, dtype: object


In [14]:
# Splitting our data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# checking our classes
data_df.label.unique()

array(['thoughts', 'substance abuse', 'stress', 'general tweet',
       'anxiety', 'awareness', 'depression'], dtype=object)

In [16]:
# creating a list varaiable for our classes
target = ['anxiety', 'awareness', 'depression', 'general tweet', 'stress', 'substance abuse','thoughts']

In [17]:
# converting our train varaibles to numpy array
X_train= X_train.to_numpy()
y_train= y_train.to_numpy()

In [18]:
# converting our test varaibles to numpy array
X_test= X_test.to_numpy()
y_test = y_test.to_numpy()

In [19]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
#tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")

#model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [25]:
# defining our model object and preprocessing our data
MODEL_NAME = "distilbert-base-uncased"
t = text.Transformer(MODEL_NAME, maxlen=500, class_names= target)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


preprocessing train...
language: en
train sequence lengths:
	mean : 9
	95percentile : 14
	99percentile : 17


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:600: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 15
	99percentile : 17


In [27]:
# Import required model class
from transformers import TFDistilBertForSequenceClassification
# Download pre-trained model
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, 
                                                              num_labels=7,
                                                              output_hidden_states=True, 
                                                              output_attentions=True
                                                              )

In [32]:
from transformers import DistilBertConfig
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512)
model = TFDistilBertForSequenceClassification(config)

In [33]:
from transformers import TFTrainer,TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=10,
    report_to="wandb",  # enable logging to W&B
    run_name="run_bal_3"
)
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")



trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=trn,         
    eval_dataset=val             
)

In [34]:
learner.fit_onecycle(1e-5, 5, callbacks=[WandbCallback()])



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
332/332 [==============================] - 230s 687ms/step - loss: 0.8225 - accuracy: 0.7555 - val_loss: 0.8471 - val_accuracy: 0.7203


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/5
332/332 [==============================] - 228s 684ms/step - loss: 0.7031 - accuracy: 0.7867 - val_loss: 0.7907 - val_accuracy: 0.7203
Epoch 3/5
332/332 [==============================] - 228s 684ms/step - loss: 0.5473 - accuracy: 0.8370 - val_loss: 0.7633 - val_accuracy: 0.7404
Epoch 4/5
332/332 [==============================] - 228s 684ms/step - loss: 0.3863 - accuracy: 0.8878 - val_loss: 0.7944 - val_accuracy: 0.7264
Epoch 5/5
332/332 [==============================] - 227s 683ms/step - loss: 0.3166 - accuracy: 0.9135 - val_loss: 0.7846 - val_accuracy: 0.7425


In [35]:
learner.validate(class_names=t.get_classes())

                 precision    recall  f1-score   support

        anxiety       0.77      0.81      0.79        68
      awareness       0.60      0.60      0.60        81
     depression       0.70      0.65      0.67        62
  general tweet       0.53      0.35      0.42        72
         stress       0.87      0.96      0.91        72
substance abuse       0.81      0.89      0.85        74
       thoughts       0.80      0.96      0.87        68

       accuracy                           0.74       497
      macro avg       0.73      0.74      0.73       497
   weighted avg       0.73      0.74      0.73       497



array([[55,  2,  6,  2,  2,  1,  0],
       [ 4, 49,  6,  7,  3,  4,  8],
       [ 5,  9, 40,  6,  1,  1,  0],
       [ 7, 17,  3, 25,  4,  8,  8],
       [ 0,  0,  2,  0, 69,  1,  0],
       [ 0,  2,  0,  6,  0, 66,  0],
       [ 0,  2,  0,  1,  0,  0, 65]])

In [36]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

----------
id:73 | loss:5.52 | true:depression | pred:substance abuse)



In [37]:
print(X_test[468])

ramsey dont stress force


In [38]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [39]:
predictor.predict('would want people around depressed pearlradioke worldsuicidepreventionday cc')

'awareness'

In [44]:
preds= predictor.predict_proba('would want people around depressed pearlradioke worldsuicidepreventionday cc')
preds

array([0.00789019, 0.87852085, 0.03921637, 0.05358986, 0.00377088,
       0.00441494, 0.01259693], dtype=float32)

In [45]:
print([target], preds)
list_arrays= ([target], preds)

[['anxiety', 'awareness', 'depression', 'general tweet', 'stress', 'substance abuse', 'thoughts']] [0.00789019 0.87852085 0.03921637 0.05358986 0.00377088 0.00441494
 0.01259693]


In [46]:
pd.DataFrame(list(map(np.ravel, list_arrays)))

,0,1,2,3,4,5,6
0,anxiety,awareness,depression,general tweet,stress,substance abuse,thoughts
1,0.00789019,0.878521,0.0392164,0.0535899,0.00377088,0.00441494,0.0125969
